In [1]:
import numpy as np
from bokeh.plotting import *
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CustomJS, Slider, Select, Button, DataTable, TableColumn, NumberFormatter
from bokeh import events
from bokeh.io import curdoc

#output_notebook()
#output_file("linked_brushing_slider_output.html", title='Bokeh demo 3')

In [2]:
# Import the data

# Define q and T
q = np.linspace(0.005, 4, num=800)
T = np.linspace(15, 70, num=12)

# Get s
s = []
fid = open("", "r") # protein saxs tempscan file
datalines = fid.readlines()
for line in datalines:
    templine = line.split("\t")
    templine[-1] = templine[-1].strip()
    s.append(templine)  
fid.close()
s = np.asarray(s)
s = s.astype(np.float)

# Get ds, scattering minus scattering at 15 C
s15mat = np.array([s[:,0],]*len(T)).transpose()
ds = s - s15mat

In [3]:
#define the different curves
s15 = s[:,0]
s20 = s[:,1]
s25 = s[:,2]
s30 = s[:,3]
s35 = s[:,4]
s40 = s[:,5]
s45 = s[:,6]
s50 = s[:,7]
s55 = s[:,8]
s60 = s[:,9]
s65 = s[:,10]
s70 = s[:,11]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(q=q, s=s15, 
                                    s15=s15, s20=s20, s25=s25, s30=s30, s35=s35, s40=s40, s45=s45,
                                    s50=s50, s55=s55, s60=s60, s65=s65, s70=s70))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
f1 = figure(tools=TOOLS, width=350, height=350, title=None, 
            x_range=(0.025, 0.7), y_range=(0.0005, 0.2),
            x_axis_type="log", y_axis_type="log")
f1.line('q', 's', source=source, color='black', line_width=2, line_alpha=1)

opts = [str(int(x)) for x in T]

i = 0
select = Select(title="Temperatures", value=opts[i], options=opts)
selectCallback = CustomJS(args=dict(source=source, select=select), code="""
    var s_data = source.data;
    var s = s_data.s;
    var s_sel = "s"+select.value;
    
    //change s array in source.data
    source.data.s = source.data[s_sel]
    
    source.change.emit();
    console.log(s_sel);
""")

select.js_on_change("value", selectCallback)


# put the subplots in a gridplot
p = gridplot([[f1]])

layout = column(
    p,
    column(select),
)
show(layout)
